In [24]:
# %%timeit
import sys
import logging

sys.path.append('..')

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType, DoubleType
from pyspark.rdd import RDD
from pyspark.sql.dataframe import DataFrame as DataFrame
from pandas.core.frame import DataFrame as PandasDataframe

# Run the date count jobs tasks
from transformations.solana import SparkJob

LOG_FILENAME = 'solana.log'
logging.basicConfig(filename=LOG_FILENAME, level=logging.INFO)


job = SparkJob()
df = job.solana_pivot()

22/04/26 16:35:11 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


preprocessing completed.
+--------------------+------+
|           open_time|volume|
+--------------------+------+
|2020-08-11 06:00:...|  6115|
|2020-08-11 06:15:...|  2813|
|2020-08-11 06:30:...|  9207|
|2020-08-11 06:45:...|  1895|
|2020-08-11 07:00:...|  5723|
|2020-08-11 07:15:...|  7339|
|2020-08-11 07:30:...| 20720|
|2020-08-11 07:45:...|  8286|
|2020-08-11 08:00:...| 13904|
|2020-08-11 08:15:...|  4970|
+--------------------+------+
only showing top 10 rows



In [25]:

df = df.select(
    'open_time', 
    current_date().alias('current_date') ,  
    date_format(current_timestamp(),"yyyy MM dd").alias("current_date_formated"),
    month('open_time').alias('month'),
    year('open_time').alias('year'),
    minute('open_time').alias('minute'),
    'volume',
    )
df.show(10)

+--------------------+------------+---------------------+-----+----+------+------+
|           open_time|current_date|current_date_formated|month|year|minute|volume|
+--------------------+------------+---------------------+-----+----+------+------+
|2020-08-11 06:00:...|  2022-04-26|           2022 04 26|    8|2020|     0|  6115|
|2020-08-11 06:15:...|  2022-04-26|           2022 04 26|    8|2020|    15|  2813|
|2020-08-11 06:30:...|  2022-04-26|           2022 04 26|    8|2020|    30|  9207|
|2020-08-11 06:45:...|  2022-04-26|           2022 04 26|    8|2020|    45|  1895|
|2020-08-11 07:00:...|  2022-04-26|           2022 04 26|    8|2020|     0|  5723|
|2020-08-11 07:15:...|  2022-04-26|           2022 04 26|    8|2020|    15|  7339|
|2020-08-11 07:30:...|  2022-04-26|           2022 04 26|    8|2020|    30| 20720|
|2020-08-11 07:45:...|  2022-04-26|           2022 04 26|    8|2020|    45|  8286|
|2020-08-11 08:00:...|  2022-04-26|           2022 04 26|    8|2020|     0| 13904|
|202

In [23]:
df.groupby("minute") \
    .pivot("month") \
    .sum('volume').show(10)

+------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|minute|       1|       2|       3|       4|       5|       6|       7|       8|       9|      10|      11|      12|
+------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|    45|72759699|61131529|43685972|56939322|59246741|48367201|24520142|51869187|75641544|37206056|34954910|28942546|
|     0|77520818|65527873|47153244|62967852|68444012|53791560|27199974|55576471|81330684|45798614|38569070|33633018|
|    30|74504056|63273295|44054054|57387350|58995975|47733935|25121176|53492405|78352750|39010925|35327033|29743231|
|    15|74007151|61964958|43937033|56883349|62698409|49322669|25446075|53030447|79780201|40973773|36300445|30047677|
+------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+



22/04/26 07:31:31 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1750466 ms exceeds timeout 120000 ms
22/04/26 07:31:31 WARN SparkContext: Killing executors is not supported by current scheduler.


In [8]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Creating a window.
window = Window.partitionBy("high").orderBy(df["volume"].desc())

df.withColumn("row_number",row_number().over(window)) \
    .show(truncate=False)

22/04/26 04:46:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----------------------------+----+----+---+-----+-------+----------+------------------+----------------+--------------+---------------+------+----------+
|open_time                    |open|high|low|close|volume |close_time|quote_asset_volume|number_of_trades|tb_base_volume|tb_quote_volume|ignore|row_number|
+-----------------------------+----+----+---+-----+-------+----------+------------------+----------------+--------------+---------------+------+----------+
|2021-01-08 10:00:00.000000000|2   |3   |2  |3    |2248151|null      |6843551           |13377           |1357068       |4114713        |0     |1         |
|2021-05-20 17:00:00.000000000|44  |53  |44 |48   |2201043|null      |107395538         |121890          |1205511       |58868213       |0     |2         |
|2021-01-08 12:30:00.000000000|3   |3   |3  |3    |1744730|null      |6238414           |12269           |956660        |3418708        |0     |3         |
|2021-01-07 10:15:00.000000000|2   |2   |2  |2    |1432337|null 

In [9]:
df3 = df.select(split(df.case1,""))
df3.show()

AttributeError: 'DataFrame' object has no attribute 'case1'

22/04/25 22:33:30 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 666749 ms exceeds timeout 120000 ms
22/04/25 22:33:30 WARN SparkContext: Killing executors is not supported by current scheduler.
